In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import hopsworks

In [4]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

### 1. IMPORT BACKFILLED DATA FROM FEATURE STORE:

In [5]:
from src.training_pipeline import taxi_fetch_data_from_feature_store

In [6]:
feat_view_name, feat_view_ver = "backfilled_2020_22", 1

df_demand_2020_22 = taxi_fetch_data_from_feature_store(feat_view_name, feat_view_ver)
df_demand_2020_22

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/571540
Connected. Call `.close()` to terminate connection gracefully.


Finished: Reading data from Hopsworks, using Hive (17.15s) 


,date_time,demand
1235,2020-01-01 00:00:00,266
14783,2020-01-01 01:00:00,231
17253,2020-01-01 02:00:00,244
22409,2020-01-01 03:00:00,259
7476,2020-01-01 04:00:00,144
...,...,...
18612,2022-12-31 19:00:00,99
26016,2022-12-31 20:00:00,101
9736,2022-12-31 21:00:00,103
14795,2022-12-31 22:00:00,63


### 2. IMPORT MODEL FORM MODEL REGISTRY:

In [7]:
from src.inference_pipeline import load_model_from_registry

In [8]:
model_name =  "model_2020_22"
model_ver = 1

trained_model = load_model_from_registry(model_name, model_ver)
trained_model

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/571540
Connected. Call `.close()` to terminate connection gracefully.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=8, ...)

### 3. MAKE FORECAST FOR NEXT MONTH:

In [9]:
from src.training_pipeline import taxi_make_forecast, taxi_display_forecaster_performance

In [10]:
past_30_days_demand = df_demand_2020_22.demand.values[-24*30:]
len(past_30_days_demand)

720

In [11]:
y_pred_jan_2023 =  taxi_make_forecast(past_30_days_demand, trained_model, forecast_horizon = 24*31)
y_pred_jan_2023.shape

(744,)

##### FORMAT PREDICTION INTO HOPSWORKS COMPATABLE FORMAT;

In [12]:
idx = pd.date_range(start = '2023-1-1', end = '2023-2-1', freq= 'h', inclusive = 'left')
idx

DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 01:00:00',
               '2023-01-01 02:00:00', '2023-01-01 03:00:00',
               '2023-01-01 04:00:00', '2023-01-01 05:00:00',
               '2023-01-01 06:00:00', '2023-01-01 07:00:00',
               '2023-01-01 08:00:00', '2023-01-01 09:00:00',
               ...
               '2023-01-31 14:00:00', '2023-01-31 15:00:00',
               '2023-01-31 16:00:00', '2023-01-31 17:00:00',
               '2023-01-31 18:00:00', '2023-01-31 19:00:00',
               '2023-01-31 20:00:00', '2023-01-31 21:00:00',
               '2023-01-31 22:00:00', '2023-01-31 23:00:00'],
              dtype='datetime64[ns]', length=744, freq='H')

In [13]:
y_pred_jan_2023 = pd.DataFrame().assign(date_time = idx, demand = y_pred_jan_2023)
y_pred_jan_2023

,date_time,demand
0,2023-01-01 00:00:00,57.896683
1,2023-01-01 01:00:00,54.317245
2,2023-01-01 02:00:00,50.874168
3,2023-01-01 03:00:00,42.993507
4,2023-01-01 04:00:00,23.888659
...,...,...
739,2023-01-31 19:00:00,96.848862
740,2023-01-31 20:00:00,91.671013
741,2023-01-31 21:00:00,79.002838
742,2023-01-31 22:00:00,73.072212


### 4. SAVE PREDICTIONS TO FEATURE STORE:

In [14]:
from src.feature_pipeline import taxi_insert_data_into_feature_store

In [15]:
data = y_pred_jan_2023
feat_group_name = 'nyc_taxi_demand_prediction_jan_2023'
feat_group_ver = 1

taxi_insert_data_into_feature_store(data, feat_group_name, feat_group_ver)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/571540
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 0.00% |          | Rows 0/744 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: nyc_taxi_demand_prediction_jan_2023_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/571540/jobs/named/nyc_taxi_demand_prediction_jan_2023_1_offline_fg_materialization/executions
data successfully inserted in feature store
